In [68]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import re
import csv
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report,precision_recall_fscore_support
# import ipwhois

In [69]:
def split_len(x):
    return len(x.split('.'))

In [70]:
df_train_benin = pd.read_csv('Data/Train_Benign_Traffic.csv',encoding="latin1")
df_train_mal = pd.read_csv('Data/Train_Detections.csv')
df_test = pd.read_csv('Data/Test_Set_Hackathon.csv')
df_train_ips = pd.read_csv('Data/Train_IPS.csv')
df_ips = pd.read_csv('Data/IPTypes.csv').set_index('ip')

In [71]:
features = df_test.columns[1:]
# easter_eggs = df[df.dst_ip.map(split_len) > 4 ].dst_ip
df_train_benin['label'] = 'benign'
df_train = pd.concat([df_train_benin,df_train_mal]).reset_index()
df_train = df_train.drop(['index','Unnamed: 0','app_name','app_risk'],axis=1)
df_train.protocol = df_train_mal.protocol.replace({'UDP':2,'TCP':1})
df_train = df_train[df_train.dst_ip.map(split_len) < 5 ].reset_index()
df_train = df_train[df_train.src_ip.map(split_len) < 5 ].reset_index().drop(['index','level_0'],axis=1)
df_train.head()

/home/yair/anaconda3/envs/CP/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


,client,dst_ip,dst_port,epoch_time,label,protocol,received_bytes,sent_bytes,src_ip
0,2,222.58.210.46,443.0,1547242292,benign,2.0,44096,67712,10.197.88.33
1,5,110.244.46.16,443.0,1547468725,benign,2.0,7712,1990,10.0.11.69
2,2,23.130.21.5,443.0,1547415821,benign,2.0,12928,8464,192.168.174.137
3,2,71.55.163.78,443.0,1547152973,benign,2.0,19808,4376,192.168.190.183
4,5,230.0.0.1,NaN,1547070189,benign,1.0,0,252,10.0.11.2


In [72]:
df_train = df_train.join(df_ips,on='src_ip',rsuffix='_src').drop('Unnamed: 0',axis=1)
df_train = df_train.join(df_ips,on='src_ip',rsuffix='_src').drop('Unnamed: 0',axis=1)
df_train = df_train.join(df_ips,on='dst_ip',lsuffix='_dst').drop(['type', 'subtype', 'Unnamed: 0'],axis=1)
df_train['log_sent_bytes'] = np.log1p(df_train.sent_bytes).astype(int)
df_train['log_received_bytes'] = np.log1p(df_train.received_bytes).astype(int)
df_train =  pd.concat([pd.DataFrame(df_train.dst_ip.apply(lambda x: x.split('.')).values.tolist(),
             columns=['dst_int_1','dst_int_2','dst_int_3','dst_int_4']),df_train],axis=1)
df_train =  pd.concat([pd.DataFrame(df_train.src_ip.apply(lambda x: x.split('.')).values.tolist(),
             columns=['src_int_1','src_int_2','src_int_3','src_int_4']),df_train],axis=1)

In [ ]:
def plot_count(series):
    counts = series.value_counts()
    plt.figure(figsize=(5,2))
    sns.barplot(counts.index,counts)

In [ ]:
def plot_pca(df):
    pca = PCA(n_components=2)
    to_pca = df.drop(['dst_ip','src_ip'],axis=1)
    to_pca = to_pca.dropna()
    labels_pca = to_pca.label
    proj = pca.fit_transform(to_pca.drop(['label'],axis=1))
    plt.figure(figsize=(10,6))
    sns.scatterplot(proj[:,0],proj[:,1],hue=labels_pca)

In [ ]:
#dummies 
df_train = pd.concat([df_train,pd.get_dummies(df_train.dst_port,prefix='port')],axis=1)
df_train = pd.concat([df_train,pd.get_dummies(df_train[['log_received_bytes','log_sent_bytes']]
                                              ,prefix=['log_received_bytes','log_sent_bytes'])],axis=1)

In [ ]:
#separate df for client

df_1 = df_train[df_train.client == 1].drop('client',axis=1)
df_2 = df_train[df_train.client == 2].drop('client',axis=1)
df_3 = df_train[df_train.client == 3].drop('client',axis=1)
df_4 = df_train[df_train.client == 4].drop('client',axis=1)
df_5 = df_train[df_train.client == 5].drop('client',axis=1)

In [ ]:
list_of_df = [df_1,df_2,df_3,df_4,df_5]

for i,df in enumerate(list_of_df):
    plot_count(df.label)
    plt.title(f'client {i+1}')
    plt.show()

In [ ]:
for i, df in enumerate(list_of_df):
    plot_pca(df)
    plt.title(f'client {i+1}')
    plt.show()

In [ ]:
prediction = []
models = []
scores = []
for df in list_of_df:
    X = df.drop(['label','src_ip','dst_ip'],axis=1)
    y = df.label
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    rf = RandomForestClassifier(class_weight='balanced')
    rf.fit(X_train,y_train)
    p = rf.predict(X_test)
    prediction.append(p)
    scores.append(precision_recall_fscore_support(y_test,p))
    models.append(rf)
    

In [ ]:
preds = df_test.dst_ip.apply(lambda x: True if x == '8.8.8.8' else False).astype(bool)

In [ ]:
ips = df_train.src_ip
ips.append(df_train.dst_ip)
pd.DataFrame(ips.unique()).to_csv('ips.csv',index=False)

In [ ]:
preds = {v:bool(k) == True for v,k in dict(preds).items()}

In [ ]:
r = requests.post('http://192.168.1.24:30000/submission',
                   json={ 'team':'DeepZuchinis',
                          'password':'4lbertKmus',
                          'submission': preds
                         },
                   verify=False)

In [ ]:
r.text